# THIS GOES SECOND. AFTER CLEANING

In [5]:
from processing_eeg_methods.data_utils import (
    get_dataset_basic_info,
)
from processing_eeg_methods.data_loaders import load_data_labels_based_on_dataset
from processing_eeg_methods.share import datasets_basic_infos

dataset_name = "braincommand"

dataset_info =get_dataset_basic_info(datasets_basic_infos, dataset_name)
data_path = r"C:\Users\rosit\Documents\workprojects\bci_complete\EEG-Classifiers-Ensemble\Datasets\braincommand_dataset"

In [6]:
import pandas as pd
import numpy as np
from scipy.io import savemat

In [7]:
def create_event_file(game_mode, y_events_original, subject_id):
    # Initialize the first latency
    prev_latency = 0
    
    # Open the file for writing
    file_name = f'C:\\Users\\rosit\\Documents\\MATLAB\\2025 clean EEG BrainCommand\\cleaned_from_python\\y_{subject_id}_{game_mode}_cleaned_events.txt'
    with open(file_name, 'w') as file:
        # Write the header
        file.write('latency\ttype\tduration\n')
        
        for y_event in y_events_original:
            # Calculate the type value
            latency = prev_latency + 1300
            
            # Write the row to the file
            file.write(f'{latency}\t{y_event}\t1300\n')
            
            # Update the latency for the next row
            prev_latency = latency
    
    print(f'File {file_name} created successfully.')

In [8]:
import os
from scipy.io import savemat

def get_player1_turns(file_path):
    player1_turns = []
    
    # Open the file and read the lines
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            if line.startswith("player1_turns"):
                # Extract and parse the player1_turns list portion
                raw_data = line.split("player1_turns,")[1].strip()
                player1_turns = eval(raw_data) # Unsafe but simple for controlled input
    
    return player1_turns

game_mode="singleplayer" # calibration3
subject_id_list = list(range(1, 27))

base_path = r"C:\Users\rosit\Documents\workprojects\bci_complete\BrainCommand\assets\game_saved_files\time_and_movement (re-numbered)"
file_pattern = "time_and_movement_singleplayer_sub{:02d}.txt"
data_output_path = r"C:\Users\rosit\Documents\MATLAB\2025 clean EEG BrainCommand"

#subject_id_list = [1]
for subject_id in subject_id_list:
    file_name = file_pattern.format(subject_id)
    file_path = os.path.join(base_path, file_name)
    player1_turns = get_player1_turns(file_path)
    
    y_fixed_numbers = [turn for sublist in player1_turns for turn in sublist]
    y_fixed_numbers = np.array(y_fixed_numbers)
    
    _, x_original, y_bad_numbers = load_data_labels_based_on_dataset(
        dataset_info,
        subject_id,
        data_path,
        game_mode=game_mode,
        normalize=False,
    )
    file_path = fr"C:\Users\rosit\Documents\MATLAB\2025 clean EEG BrainCommand\datasets_edited\{game_mode}\{subject_id}_{game_mode}_cleaned_events.txt"
    df = pd.read_csv(file_path, sep='\t')
    df['urevent'] = df['urevent'].astype(int)
    indexes = [x - 1 for x in list(df['urevent'])]
    
    y_cleaned = y_fixed_numbers[indexes]
    x_cleaned = x_original[indexes, :,  :] # trials x channel x time

    create_event_file(game_mode, y_cleaned, subject_id) # to load it in eeglab

    x_cleaned_t = np.transpose(x_cleaned, (1, 2, 0)) # output: channel x sample x trial
    data_dict = {"array": x_cleaned_t}
    data_output_path = r"C:\Users\rosit\Documents\MATLAB\2025 clean EEG BrainCommand"
    savemat(fr"{data_output_path}\cleaned_from_python\{subject_id}_{game_mode}_cleaned.mat", data_dict)
    
    labels_dict = {"tags": y_cleaned}
    savemat(fr"{data_output_path}\cleaned_from_python\y_{subject_id}_{game_mode}_cleaned.mat", labels_dict)
    

label 0 is 100
label 1 is 103
label 2 is 51
label 3 is 22
Not setting metadata
276 matching events found
Setting baseline interval to [0.0, 1.296] s
Applying baseline correction (mode: mean)
0 projection items activated
File C:\Users\rosit\Documents\MATLAB\2025 clean EEG BrainCommand\cleaned_from_python\y_1_singleplayer_cleaned_events.txt created successfully.
label 0 is 85
label 1 is 102
label 2 is 65
label 3 is 23
Not setting metadata
275 matching events found
Setting baseline interval to [0.0, 1.296] s
Applying baseline correction (mode: mean)
0 projection items activated
File C:\Users\rosit\Documents\MATLAB\2025 clean EEG BrainCommand\cleaned_from_python\y_2_singleplayer_cleaned_events.txt created successfully.
label 0 is 81
label 1 is 102
label 2 is 24
label 3 is 48
Not setting metadata
255 matching events found
Setting baseline interval to [0.0, 1.296] s
Applying baseline correction (mode: mean)
0 projection items activated
File C:\Users\rosit\Documents\MATLAB\2025 clean EEG Brai